# Download WFS Data
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lalgonzales/datasets/blob/main/notebooks/download_wfs.ipynb)

This notebook is used to generate the datasets shared in this repository. It demonstrates how to download data from a Web Feature Service (WFS) and save it in various formats such as Shapefile, GeoJSON, and GPKG. The downloaded shapefile data is then compressed into ZIP files for easy distribution.

## Explore and Download Data

Feel free to explore and download the data using this notebook on your local machine or in Google Colab. You can modify the notebook to suit your needs and generate the datasets as required.

# 1. Libraries


In [ ]:
# !pip install owslib

In [2]:
import geopandas as gpd
import os
import zipfile
import shutil
import warnings
from owslib.wfs import WebFeatureService


# 2. Get the data

In [ ]:
# Create output vector directory if it doesn't exist
output_dir = 'vector'
os.makedirs(output_dir, exist_ok=True)


In [ ]:
# Get the WFS capabilities
url = "https://geoserver.icf.gob.hn/icf/wfs"
wfs = WebFeatureService(url=url, version='1.1.0')

# Create a list of the layers available
layers = list(wfs.contents)
layers


In [ ]:
# Create a dictionary naming the layers and their titles
layer_dict = {}
for layer in layers:
    title = wfs[layer].title
    layer_dict[title] = layer
layer_dict


In [5]:
# Define the list of layers to download
layer_list = [
    "Limite_HN", # National boundary of Honduras
    "adef", # Deforestation alerts from Global Forest Watch
    "areas_asignadas", #
    "areas_protegidas", # Declared protected areas
    "areas_protegidas_propuesta", # Proposed protected areas
    "m1102va001970_hn", # Departametnal boundaries
    "m1103va002001_hn", # Municipal boundaries,
    "m1104vA002001_HN", # Village boundaries
    "microcuencas_declaradas", # Declared watersheds
    "planes_de_manejo_aprobados", # Approved forest management plans
    "regiones_forestales_icf", # ICF forest regions
    "reservas_biosfera", # Biosphere reserves
    "reservas_naturales_privadas", # Private nature reserves
    "vida_silvestre", # Wildlife sites
]


In [6]:
# Function to rename layers
def rename_layer(layer):
    layer_names = {
        'adef': 'alertas_deforestacion',
        'm1102va001970_hn': 'departamentos_hn',
        'm1103va002001_hn': 'municipios_hn',
        'm1104vA002001_HN': 'aldeas_hn'
    }
    return layer_names.get(layer, layer)


In [7]:
# Check if layers exist in WFS
available_layers = wfs.contents.keys()
missing_layers = [layer for layer in layer_list if layer_dict[layer] not in available_layers]

if missing_layers:
    print(f"Warning: The following layers are missing in the WFS and will be skipped: {missing_layers}")


In [ ]:
# Download the layers as geojson files
print("Downloading layers...")
for layer in layer_list:
    if layer_dict[layer] not in available_layers:
        continue
    try:
        response = wfs.getfeature(typename=layer_dict[layer], outputFormat="json")
        renamed_layer = rename_layer(layer)
        with open(os.path.join(output_dir, f"{renamed_layer}.geojson"), "wb") as file:
            file.write(response.read())
        print(f"{renamed_layer} downloaded as GeoJSON")

    except Exception as e:
        print(f"Error processing layer {layer}: {e}")

print(f"Downloaded {len(layer_list) - len(missing_layers)} layers as GeoJSON files")


In [ ]:
# Download the layers as gkpg files
print("Downloading layers...")
for layer in layer_list:
    if layer_dict[layer] not in available_layers:
        continue
    try:
        response = wfs.getfeature(typename=layer_dict[layer], outputFormat="json")
        gdf = gpd.read_file(response)
        renamed_layer = rename_layer(layer)
        # Save temporary GPKG
        gpkg_path = os.path.join(output_dir, f"{renamed_layer}.gpkg")
        gdf.to_file(gpkg_path, driver="GPKG")
        print(f"{renamed_layer} downloaded as GPKG")

    except Exception as e:
        print(f"Error processing layer {layer}: {e}")

print(f"Downloaded {len(layer_list) - len(missing_layers)} layers as GPKG files")


In [ ]:
# Suprimir advertencias
warnings.filterwarnings("ignore", category=UserWarning, module='geopandas')
warnings.filterwarnings("ignore", category=RuntimeWarning, module='pyogrio')

# Create temporary directory
tmp_dir = 'tmp_shapefile'
os.makedirs(tmp_dir, exist_ok=True)

# Download the layers as shapefiles and zip them
print("Downloading layers...")
for layer in layer_list:
    if layer_dict[layer] not in available_layers:
        continue
    try:
        response = wfs.getfeature(typename=layer_dict[layer], outputFormat="json")
        gdf = gpd.read_file(response)
        renamed_layer = rename_layer(layer)
        # Save temporary shapefile
        layer_tmp_dir = os.path.join(tmp_dir, renamed_layer)
        os.makedirs(layer_tmp_dir, exist_ok=True)
        gdf.to_file(f"{layer_tmp_dir}/{renamed_layer}.shp")
        print(f"{renamed_layer} downloaded as shapefile")

        # Zip the shapefile
        shapefile_zip = os.path.join(output_dir, f"{renamed_layer}.zip")
        with zipfile.ZipFile(shapefile_zip, 'w') as zipf:
            for root, dirs, files in os.walk(layer_tmp_dir):
                for file in files:
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), layer_tmp_dir))

        print(f"{renamed_layer} shapefile zipped as {shapefile_zip}")
    except Exception as e:
        print(f"Error processing layer {layer}: {e}")

# Clean up temporary directory
shutil.rmtree(tmp_dir)

print(f"Downloaded and zipped {len(layer_list) - len(missing_layers)} layers")
